# **PASOS**

**1. Crear/reutilizar los siguientes archivos (exactamente el mismo nombre):**

a. cuerpo.txt: aqui poner contenido del mail (recordar que NOMBRE_EMPRESA se cambiara por el nombre de la empresa en cuestion) (obligatorio)

b. contraseña.txt aqui va la contraseña generada en gmail (obligatorio)

c. contactados mails.txt: aqui se guardaran los mails contactados y permitira el control de envios (envios maximos para cada mail) (opcional).

d. contactados nombres.txt: aqui se guardaran el nombre de las empresas contactadas (opcional)


**2. Subirlos a este codigo (clickear la carpetita arriba a la izq y arrastrarlos alli)**

**3. Definir al comienzo del codigo:**

a. pagina: hasta que pagina buscar en Clutch e Indeed

b. limite_envios: cantidad de envios maximos a cada mail (si nunca se subio el archivo contactados mails.txt, entonces esto no importa porque se tomara como que no hay limite)

c. asunto: el asunto del mail (recordar que NOMBRE_EMPRESA se cambiara por el nombre de la empresa en cuestion)

d. email_from: email desde el cual se enviaran los mails (si se cambia el email, repetir el paso para obtener la contraseña)

# **RESULTADO**

1. Se enviaran mails a todas las empresas posibles.
2. En caso de subir contactados mails.txt, se descargara el mismo archivo con todos los nuevos mails de las empresas contactadas
3. En caso de subir contactados nombres.txt, se descargara el mismo archivo con todos los nuevos nombres de las empresas contactadas

In [ ]:
pagina = 1
limite_envios = 10
asunto = 'Trabajar en NOMBRE_EMPRESA'
email_from = 'ejemplojoaco@gmail.com'










from bs4 import BeautifulSoup
import requests
from time import sleep
import re
import pandas as pd
import numpy as np
from itertools import chain
from email.message import EmailMessage
import ssl
import smtplib
from google.colab import files


# Headers
headers = {
    'authority': 'ms-mt--api-web.spain.advgo.net',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/plain, */*',
    'x-adevinta-channel': 'web-desktop',
    'x-schibsted-tenant': 'coches',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'content-type': 'application/json;charset=UTF-8',
    'origin': 'https://www.coches.net',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.coches.net/',
    'accept-language': 'en-US,en;q=0.9,es;q=0.8',
}

# Buscador emails
def buscadorEmails(all_links):

  # Decodificador mails encriptados
  def decodeEmail(e):
      de = ""
      k = int(e[:2], 16)
      for i in range(2, len(e)-1, 2):
          de += chr(int(e[i:i+2], 16)^k)
      return de

  # Encontrar emails directamente
  condicion = "[a-zA-Z0-9\.\-+_]+@[a-zA-Z0-9\.\-+_]{4,}" # regex email
  contactos = ['Contact','contact','CONTACT'] # palabras a buscar dentro de la pagina
  prohibidos = ['.jpg','.png']
  emails = []

  # Iteracion por paginas web
  contador = 0
  for link in all_links:
    try:
      contador += 1
      response = requests.get(link,headers=headers)
      new_emails = list(set(re.findall(condicion, 
                      response.text, re.I)))
      # Encontrar los encriptados
      soup = BeautifulSoup(response.content, 'html.parser')
      mailsEncriptados = soup.find_all("span", class_="__cf_email__")
      for me in mailsEncriptados:
        mailLiberado = re.findall('data-cfemail="([a-zA-Z0-9\.\-+_]+)"', str(me), re.I)
        new_emails.extend([decodeEmail(mailLiberado[0])])
      # Encontrar emails a traves de "contacto"
      try:
        links1 = []
        response1 = requests.get(link,headers=headers)
        soup1 = BeautifulSoup(response1.content, 'html.parser')
        links1 = soup1.find_all('a')
        links1 = [link.get('href') for link in links1 for c in contactos if c in link.get('href')]
        new_emails1 = []
        for l in links1:
          try:
            responseContacto = requests.get(l,headers=headers)
            new_emails1.extend(list(set(re.findall(condicion, 
                            responseContacto.text, re.I))))
          except:
            pass
        new_emails.extend(new_emails1)
        try:
            # Encontrar los encriptados
          soup2 = BeautifulSoup(responseContacto.content, 'html.parser')
          mailsEncriptados = soup2.find_all("span", class_="__cf_email__")
          for me in mailsEncriptados:
            mailLiberado = re.findall('data-cfemail="([a-zA-Z0-9\.\-+_]+)"', str(me), re.I)
            new_emails.extend([decodeEmail(mailLiberado[0])])
        except:
          pass
      except:
        pass

      # Limpieza
      new_emails = [ne.split(',') for ne in new_emails] # separar los emails pegados
      if len(new_emails)>0:
        while type(new_emails[0]) == list: # desanidar las listas
          new_emails = list(chain(*new_emails))
        new_emails = [ne for ne in new_emails for p in prohibidos if p not in ne] # quitar los prohibidos en listas
        new_emails = [ne for ne in new_emails if len(re.findall('@[a-zA-Z]+',ne))>0] # aseguramos que haya al menos una letra luego del arroba
        new_emails = list(set(new_emails)) # quitar repetidos
        emails.append(new_emails)
      else:
        emails.append('sinEmail')
      print(f'Buscando mails de empresa {contador} de {len(all_links)}\n')
    except:
      emails.append('sinEmail')
  return emails

# Buscar mails
all_links = []
all_names = []
all_buscadorWeb = []
buscador = []
emails = []

# Clutch
# Nombre y link a pag web
# Iteracion por paginas de Clutch
for i in range(pagina):
  try:
    brandUrl = 'https://clutch.co/agencies/graphic-designers?page='+str(pagina+1)
    response = requests.get(brandUrl,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    companys = soup.find_all('div', class_='row')
    # Iteracion por empresas
    for company in companys: 

      # Nombre empresa
      names = company.find_all('h3', attrs={'class': 'company_info'})
      names = [tag.text.strip() for tag in names]

      # Pagina web
      web = company.find_all('a', href=True)
      web = [a['href'] for a in web if 'http' in a['href'] and 'clutch.co/profile' not in a['href']]
      
      # Contacto a travez de Clutch
      buscadorWeb = company.find_all('a', href=True)
      buscadorWeb = [a['href'] for a in buscadorWeb if 'http' in a['href'] and 'clutch.co/profile' in a['href']]
      
      # Chequea que haya encontrado si o si todos los datos
      if len(names) > 0 and len(web) > 0 and len(buscadorWeb) > 0:
        all_names.extend(names)
        all_links.extend(web)
        all_buscadorWeb.extend(buscadorWeb)
        buscador.append('Clutch')
  except:
    pass

# Indeed
# Nombre y link a pag web
for i in range(pagina):
  try:
    brandUrl = 'https://www.indeed.com/jobs?q=animator&start='+str(i)+'0'
    response = requests.get(brandUrl,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Name
    companysName = soup.find_all('span', class_='companyName')
    companysName = [cn.text for cn in companysName]

    # Pagina web
    base = 'https://www.indeed.com'
    companysWeb = soup.find_all('a', class_='jcs-JobTitle')
    companysWeb = [base+re.findall('href="(.+?)"', str(cw), re.I)[0] for cw in companysWeb]

    # Contacto a travez de indeed.
    base = 'https://www.indeed.com/viewjob?jk='
    buscadorWeb = soup.find_all('a', class_='jcs-JobTitle')
    buscadorWeb = [base+re.findall('data-jk="(.+?)"', str(cw), re.I)[0] for cw in buscadorWeb]

    # Chequea que haya encontrado si o si todos los datos
    if len(companysName) == len(companysWeb) and len(companysWeb) == len(buscadorWeb):
      all_names.extend(companysName)
      all_links.extend(companysWeb)
      all_buscadorWeb.extend(buscadorWeb)
      buscador.extend(['Indeed']*len(companysName)) 
  except:
    pass

# Busqueda de todos los emails   
emails = buscadorEmails(all_links)


# df
df = pd.DataFrame(np.array([all_names,emails,all_links,buscador,all_buscadorWeb]).T,columns=['Empresa','Email','web','buscador','contactoBuscador'])
dfInvalidos = df[df['Email']=='sinEmail']
df.drop(dfInvalidos.index, inplace=True)
df.reset_index(inplace=True,drop=True)


# Mandado de mail
# cuerpo
with open('cuerpo.txt') as f:
    cuerpo = f.readlines()
cuerpo = ' '.join(cuerpo)

# contraseña
with open('contraseña.txt') as f:
    email_password = f.readlines()[0]

# Ver si hay limite de contactadosMails
documentos = !ls
documentos = ' '.join(documentos)
contactadosMailsBool = [True if 'contactados mails' in documentos else False][0]
contactadosNombreBool = [True if 'contactados nombre' in documentos else False][0]

# contactados
if contactadosMailsBool:
  with open('contactados mails.txt') as f:
      contactadosMails = f.readlines()
  contactadosMails = [c.replace('\n','') for c in contactadosMails]

# Iteracion mandadora emails
contactadosNombre = []
for i in range(len(df)):

  # email receptores
  email_to = df.Email[i]
  
  # reemplazo en cuerpo
  cuerpoFinal = cuerpo.replace('NOMBRE_EMPRESA',df.Empresa[i])

  # reemplazo en asunto
  asuntoFinal = asunto.replace('NOMBRE_EMPRESA',df.Empresa[i])

  # iteterar por emails
  for et in email_to:
      # ver si hay limite de envios
      if contactadosMailsBool:
        if contactadosMails.count(et) <= limite_envios:

          # contenido
          em = EmailMessage()
          em['From'] = email_from
          em['To'] = et # et
          em['Subject'] = asuntoFinal
          em.set_content(cuerpoFinal)
          context = ssl.create_default_context()
          # Mandarlo
          with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
            smtp.login(email_from, email_password)
            smtp.sendmail(email_from, et, em.as_string())
          # print
          print(f'Mandando mail a {df.Empresa[i]} ({i+1} de {len(df)})\n')
          # contactadosMails
          contactadosMails.append(et)
          # contactadosMails nombre si existe
          if contactadosNombreBool:
            contactadosNombre.append(df.Empresa[i])
          # sleep
          sleep(3)
        else:
          print('no entro')
      else: # siempre mandarlo
        # contenido
        em = EmailMessage()
        em['From'] = email_from
        em['To'] = et
        em['Subject'] = asuntoFinal
        em.set_content(cuerpoFinal)
        context = ssl.create_default_context()
        # Mandarlo
        with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
          smtp.login(email_from, email_password)
          smtp.sendmail(email_from, et, em.as_string()) 
        # print
        print(f'Mandando mail a {df.Empresa[i]} ({i+1} de {len(df)})\n')
        # contactados nombre si existe
        if contactadosNombreBool:
          contactadosNombre.append(df.Empresa[i])
        # sleep
        sleep(3)


# Guardado contactadosMails
if contactadosMailsBool:
  contactadosMails = sorted(contactadosMails)
  with open("contactados mails.txt", "w") as output:
      output.write('\n'.join(contactadosMails))
  files.download('contactados mails.txt')

# Guardado contactadosMails nombre
if contactadosNombreBool:
  contactadosNombre = sorted(contactadosNombre)
  with open("contactados nombres.txt", "w") as output:
      output.write('\n'.join(contactadosNombre))
  files.download('contactados nombres.txt')